# Previous Work

The previous net saved in _'01-07-17_convnet.h5'_ , gets a 97% accuracy rate on the test dataset (!) of GTSRB. However, when it must predict on images that don't belong to the GTSRB dataset, it sees traffic signs where there are none, and with very high (~100%) confidence. This is a problem because during a race, the car won't see any traffic signs most of the time.

The approach to fix this, is to add a _background_ class, also called _Zero_ class, as the 43th class to the dataset.

In [ ]:
#Load Picture data functions + Load Numpy and Plot library
from utilities.gtsrb_loader.load_data import load_data
from utilities.gtsrb_loader.get_folderpath import get_folderpath
import matplotlib.pyplot as plt
import random
from datetime import date
import os
from keras.models import load_model
import numpy as np

In [ ]:
#Resizing images to 64 x 64
from skimage.transform import resize
X_train_zero = np.array([resize(pic, (64, 64), mode='edge') for pic in X_train_zero])

# Training Set Selection

For this second net, we want to distinguish between _background_ and _traffic sign_ reliably. Thats why I dont balance number of training images per class yet. Any traffic sign does the job. Since our zero class is only in gray color, we will transfor the GTSRB data to grayscale

# IF area: 
## Choose Either Black and White or Color

# Black and White Zero class

In [ ]:
#Generate Black and white pictures (First just for class 29 to test)
from skimage import color
x = color.rgb2gray(X_train[0])
print (y_train[0])
print ("Matrix x: (Anzahl Zeilen, Anzahl Spalten)", x.shape)

In [ ]:
#Show generated grey picture X_train[0]
plt.subplot(1, 2, 1)
plt.imshow(X_train[0])
plt.subplot(1, 2, 2)
plt.imshow(x, cmap='gray')
plt.show()

In [ ]:
#Generate Black and white pictures for all classes
X_train_gray = np.array([color.rgb2gray(pic) for pic in X_train])

In [ ]:
print (X_train_zero.shape, X_train_gray.shape) #Q: What is the purpose of this line?

In [ ]:
#Pictures get edited in Preprocessing to generate a bigger amount of data

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(  #Adjust datagenerator
    featurewise_center=True, #Set input mean to 0 over the dataset, feature-wise 
    #Q: What does this line mean?
    featurewise_std_normalization=True, #Divide inputs by std of the dataset, feature-wise. 
    #Q: What does this line mean?
    rotation_range=20, #Pictures get rotated (max. 20°)
    width_shift_range=0.2, #Pictures get shifted horizontally (max. by the factor 0.2)
    height_shift_range=0.2, #Pictures get shifted vertically (max. by the factor 0.2)
    horizontal_flip=True) #Pictures get mirrored horizontally 
    #Q: Is this executed with all signs?

In [ ]:
X_train_zero = X_train_zero.reshape(len(X_train_zero), 64, 64, 1) # grayscale wrapper for the datagen object 
print (X_train_zero.shape)

In [ ]:
datagen.fit(X_train_zero) #Compute the internal data stats related to the 
#data-dependent transformations, based on an array of sample data.
#Only required if featurewise_center or featurewise_std_normalization

In [ ]:
X_train_gray.max(), X_train_gray.min() #Frage: Was macht diese Funktion?

# Color zero class
since we have a colorfull zeroclass we do not need to transform the pictures to grey


In [ ]:
#Load colorful signs
#path = get_folderpath(subset='train', original_images=False)
path="/data_on_server/Images/GTSRB_64x64/Final_Training"
X_train, y_train= load_data(path)


In [ ]:
#Load colorful zero classes
#path_zero = "Y:\\Zero_Class_color\\00043"
path_zero = "//data_on_server//Images//Zero_Class_color"
X_train_zero, y_train_zero = load_data(path_zero)

In [ ]:
#The whole dataset consists of the zeroclasses as well as the normal signs
X_trainC=X_train+X_train_zero
Y_trainC=y_train+y_train_zero

In [ ]:
shuff=list(zip(X_trainC,Y_trainC))
random.shuffle(shuff)
X_trainC, Y_trainC=zip(*shuff)

In [ ]:
#List the trainingimages in a numpy array
sign=np.array(Y_trainC)
unique_classes = len(np.unique(sign))
y_trainN = np.eye(unique_classes)[np.array(sign, dtype=int)]
X_trainN = np.array(X_trainC, dtype=np.float32) / 255

In [ ]:
del X_train, X_train_zero, X_trainC
del y_train, y_train_zero, Y_trainC
del shuff

In [ ]:
#Load Testdata:
path="/data_on_server/Images/GTSRB_64x64/Final_Test"
X_test, Y_test = load_data(path)

In [ ]:
#List the testimages in a numpy array
sign_test=np.array(Y_test)
unique_classes_test = len(np.unique(sign_test))+1
Y_testN = np.eye(unique_classes_test)[np.array(sign_test, dtype=int)]
X_testN = np.array(X_test, dtype=np.float32) / 255

In [ ]:
#Identify all Trainingimages which have not the specified format
for n in X_trainN:
    if(n.shape != (64,64,3)):
        print(n.shape)

In [ ]:
#Number of training images
print (np.array(y_trainN).shape)
print (X_trainN.shape)

In [ ]:
#grayscale wrapper for the datagen object #Q: What should be done here?
X_train = X_trainN.reshape(len(X_train), 64, 64, 3)
X_train.shape

# Load Network:
From Keras docs, VGG-like Convnet.

First Block (Convolutional Block)

In [ ]:
#Other Callbacks: #Q: Where are they used?
from keras.callbacks import Callback
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

#Q: What is this/ Is this training function used?
model.fit(X_trainN, y_trainN, batch_size=32, epochs=50, validation_data=(X_testN, Y_testN), shuffle=True, callbacks=[TestCallback((X_testN, Y_testN))])

load model:

In [ ]:
model = load_model('/data_on_server/Nets/untrained/2017-11-19_convnet24_UN.h5')

for 3D-Conv Layer at start:

In [ ]:
length=len(X_trainN)
np.reshape(X_trainN,(length,1,64,64,3))
np.reshape(X_trainN,(length,1,64,64,3))


## Train Model

In [ ]:
#Tensorboard
from keras.callbacks import TensorBoard
basefolder='/data_on_server/Nets/trained/'
netname=date.today().__str__() + '_Test'
logfolder=basefolder+netname
if not os.path.exists(logfolder):
    os.makedirs(logfolder)
tensorboard = TensorBoard(log_dir=logfolder, #Path of directory where to save the log files to be parsed by TensorBoard.
                          histogram_freq=1, #frequency (in epochs) at which to compute activation and weight histograms
                                            #If set to 0, histograms won't be computed.
                          write_graph=True, #Visualize the graph in TensorBoard.
                          write_images=False) #Write model weights to visualize as image in TensorBoard.

In [ ]:
#Train model with tensorborad
h=model.fit(X_trainN, y_trainN, batch_size=32, epochs=50, validation_split=0.25, verbose=1, shuffle=True, callbacks=[tensorboard])

In [ ]:
#Save model
model.save(basefolder+netname+'.h5')

